# Reproduce channeling calculation from B. Doyle's excel file
B. doyle has an excel file all calculations are based on this excel file which in turn seems to be based on the IBA Handbook.

##Lets define some inputs
Probably these will need to be replaced by some data from files

In [76]:
%matplotlib inline
from scipy.constants import *
import matplotlib.pyplot as plt
from numpy import *
from mendeleev import element
import pandas as pd

Create a pandas data structure that holds some additional data about the elements

The Depye temperatures (in kelvin) come from
http://hyperphysics.phy-astr.gsu.edu/hbase/tables/thrcn2.html
Which has taken the values from Kittel, Charles, Introduction to Solid State Physics, 7th Ed., Wiley, (1996).

In [77]:
Z=[i for i in range(element('Hydrogen').atomic_number,element('Uranium').atomic_number)]
Symbol=dict(zip(Z,[element(i).symbol for i in range(element('Hydrogen').atomic_number,element('Uranium').atomic_number)]))
Name=dict(zip(Z,[element(i).name for i in range(element('Hydrogen').atomic_number,element('Uranium').atomic_number)]))
T_D2={3:344,4:1440,6:2230,10:75,
      11:158,12:400,13:428,14:645,18:92,
      19:91,20:230,21:360,22:420,23:380,24:630,25:410,26:470,27:445,28:450,29:343,30:327,31:320,32:374,33:282,34:90,36:72,
      37:56,38:147,39:280,40:291,41:275,42:450,44:600,45:480,46:274,47:225,48:209,49:108,50:200,51:211,52:153,54:64,
      55:38,56:110,57:142,64:200,66:210,70:120,71:210,72:252,73:240,74:400,75:430,76:500,77:420,78:240,79:165,80:71.9,81:78.5,82:105,83:119,86:64,
      90:163,92:207
     }
Elements_data=pd.DataFrame({'Symbol': Symbol,'Name':Name,'T_D2':T_D2})

In [78]:
Target=element('Au')
Projectile=element('He')
Z1= Projectile.atomic_number# He
Z2=Target.atomic_number # Au
Energy=300000 # V needs to be converted to J later on
Orientation=100
Temperature=293 # K
crystal_structure=Target.lattice_structure
print(crystal_structure)
# Or use the following to override
# Possible optiosn are FCC, BCC or DIA
#crystal_structure='fcc'
Miller={'h':1,'k':1,'l':1}

FCC


In [79]:
M2=Target.mass # amu
Rho2=Target.density # g/cm³
T_D2=Elements_data.get_value(Z2,'T_D2') # K
# Our database contains 165 K for Au BD uses 170 K so we manually reset this for the moment
T_D2=170
M1=Projectile.mass # amu

The following converts the lattice constant into the d spacing for the desired crystal orientation. this should work at least for the cubic ones. For this to work we read in the lattice factors given by BD.

In [80]:
# These are the command used to crate the pickle data from BSs Excel file. The excel file has been saved as csv and scrubbed from useless cells by hand
#diamond_lattice=pd.read_csv('diamond-lattice-factor.csv',usecols=(1,2,3,4,5,6),header=None,names=('h','k','l','# atoms','fa','fp'))
#fcc_lattice=pd.read_csv('fcc-lattice-factor.csv',usecols=(1,2,3,4,5,6),header=None,names=('h','k','l','# atoms','fa','fp'))
#bcc_lattice=pd.read_csv('bcc-lattice-factor.csv',usecols=(1,2,3,4,5,6),header=None,names=('h','k','l','# atoms','fa','fp'))
#diamond_lattice.to_pickle('Diamond-lattice.pkl')
#fcc_lattice.to_pickle('FCC-lattice.pkl')
#bcc_lattice.to_pickle('BCC-latice.pkl')
fcc_lattice=pd.read_pickle('FCC-lattice.pkl')
bcc_lattice=pd.read_pickle('BCC-lattice.pkl')
diamond_lattice=pd.read_pickle('Diamond-lattice.pkl')

Now find fa and fb for the correct lattice type and orientation

In [81]:
if crystal_structure=='FCC':
    fa=fcc_lattice.loc[(fcc_lattice['h'] == Miller['h'])&(fcc_lattice['k'] == Miller['k'])&(fcc_lattice['l'] == Miller['l']),'fa'].iloc[0]
    fp=fcc_lattice.loc[(fcc_lattice['h'] == Miller['h'])&(fcc_lattice['k'] == Miller['k'])&(fcc_lattice['l'] == Miller['l']),'fp'].iloc[0]
    print(fa,fp)
elif crystal_structure=='BCC':
    fa=bcc_lattice.loc[(bcc_lattice['h'] == Miller['h'])&(bcc_lattice['k'] == Miller['k'])&(bcc_lattice['l'] == Miller['l']),'fa'].iloc[0]
    fp=bcc_lattice.loc[(bcc_lattice['h'] == Miller['h'])&(bcc_lattice['k'] == Miller['k'])&(bcc_lattice['l'] == Miller['l']),'fp'].iloc[0]
    print(fa,fp)
elif crystal_structure=='DIA':
    fa=diamond_lattice.loc[(diamond_lattice['h'] == Miller['h'])&(diamond_lattice['k'] == Miller['k'])&(diamond_lattice['l'] == Miller['l']),'fa'].iloc[0]
    fp=diamond_lattice.loc[(diamond_lattice['h'] == Miller['h'])&(diamond_lattice['k'] == Miller['k'])&(diamond_lattice['l'] == Miller['l']),'fp'].iloc[0]
    print(fa,fp)
else:
    print('The crystal structure '+crystal_structure+' is unkown.')


1.732050808 0.577350269


In [82]:
d=Target.lattice_constant*fa # in A
print(d)

7.06676729664


# Calculate the TF screening length. 
According to BD this comes from the IBA handbook p645. Still need to figure out which IBA handbook he is refering to.
He provides three different versions:
$$a_1=0.04685(\sqrt Z_1+\sqrt Z_2)^{-2/3}$$ which he shortens to $a_{1s}=0.4685Z_2^{-1/3}$. According to BD $$a_{BD}=0.04685(Z_1^{2/3}+Z_2^{2/3})^{-1/2}$$ provides better results. There is an issue here with the values he quotes (0.04684) and what he uses for calculations (0.4685). Which is closer to the actual value of $a_0=0.4692$.
My own version would be the Thomas Fermi screening length from Nastasi book equ. 4.3
$$a_{TF}=\frac{0.885 a_0}{(Z_1^{1/2}+Z_2^{1/2})^{2/3}}$$
where $a_0=0.053$nm is the Bohrradius.

BD uses Angstrom for the Bohr Radius and not nm.

In [83]:
a0=(9*pi**2/128)**(1./3.)*0.53
a1=0.4685*(sqrt(Z1)+sqrt(Z2))**(-2./3.)
a1s=0.4685*Z2**(-1./3.)
aBD=a0*(Z1**(2./3.)+Z2**(2./3.))**(-1./2.)
Bohr_radius=codata.value('Bohr radius')
aTF=0.885*Bohr_radius/(Z1**(0.5)+Z2**(0.5))**(2./3.)
print ('aBD = {0:.5f} nm' .format(aBD)) # we go with the BD value for now

aBD = 0.10493 nm


Next is the Debye function it seems it is based on p467 from the mysterious black IBA book. Equation A15.2 from this book (?) provides the factor 
$$u_1=12.1\left(\left(\frac{\Phi_D(x")}{x"}+0.25\right)\left(\frac{1}{M_2\Theta_D}\right)\right)^{1/2}$$
where $\Phi_D=ae^{-x"/b}$ with $b=4.3$, $a=1$ and $x"=\frac{\Theta_D}{T}$. There is a prefactor that I do not understand but the result is Angstrom.

In [84]:
a=1
b=4.3
x2=T_D2/Temperature
PhiD=a*exp(-x2/b)
print('Phi_D = {0:.5f}' .format(PhiD))
u1=12.1*((PhiD/x2+0.25)*(1./(M2*T_D2)))**0.5
print('u1 = {0:.5f} A' .format(u1))

Phi_D = 0.87378
u1 = 0.08762 A


Now we can calculate $$\Psi_1=\sqrt\frac{2Z_1Z_2e^2}{4\pi\epsilon_0Ed}$$ in rad. As usual in his excel sheet 4$\pi\epsilon_0$ are missing.

In [85]:
q=codata.value('elementary charge')
eps_0=codata.value('electric constant')
psi1=sqrt((2*Z1*Z2*q**2)/((4*pi*eps_0)*Energy*q*d*1e-10))
print('Psi1 = {0:.5f}°'.format(degrees(psi1)))

Psi1 = 2.65443°


Next depending on the ratio $a_{TF}/d$ to $\Psi_1$ we calculate $\Psi_{1/2}$
If $$\Psi_1(rad)<\frac{a_TF}{d}\ldots\Psi_{1/2}=0.87F_{RS}(x')\Psi_1$$ 
or
$$\Psi_1(rad)>\frac{a_TF}{d}\ldots\Psi_{1/2}=7.57\sqrt\frac{a_{TF}\Psi_1}{d}$$
with $a=1.9$, $b=1.2$ and $c=0.53$ and 
$$x'=1.2\frac{u_1}{a_{TF}}$$ and 
$$F_{RS}(x')=ae^{-\frac{x'^c}{b}}$$

In [86]:
a=1.9
b=1.2
c=0.53
x1=1.2*u1/aBD # u1 is in angstrom so we need to do the same for aTF. for the moment we will stick to a0
FRS=a*exp(-(x1**c/b))
print('x\' = {:0.5f}'.format(x1))
print(FRS)

x' = 1.00212
0.824964027463


In [87]:
if psi1<aBD/d: # again we use aBd; this compares to the radian value
    psi12=0.87*FRS*degrees(psi1) #here we use degree for psi1
else:
    psi12=7.57*sqrt(aBD*degrees(psi1)/d) #here we use degree for psi1
print('Psi 1/2 = {:0.5f}'.format(psi12))

Psi 1/2 = 1.50284


Lets do the same for the planar channeling. Most of the stuff is already in place, and can be reused from above. We strat with calculating the new interplanar spacing $d$.

In [89]:
d=Target.lattice_constant*fp
print(d)

2.35558909752


To calculate the planar channeling angle 
$$\Psi_a=0.545\sqrt{\frac{Z_1Z_2Nd_Pa_{TF}}{E}}$$ 
in degrees we need the number of atomic density 
$$N=\frac{\rho NA}{M_2}$$ in $atoms/\angstrom^2$.

In [101]:
N=1e-24*Rho2*codata.value('Avogadro constant')/M2 #atoms/A³
Psi_a=0.545*sqrt((Z1*Z2*N*d*aBD)/(Energy*1e-6)) # The enrgy needs to be in MeV
print('Psi_a = {:0.5f}'.format(Psi_a)+'°')

Psi_a = 1.51048°


Now again some stuff that needs to be checked with the black IBA handbook. $$x'=1.6\frac{u_1}{a_{TF}}$$
and
$$y'=\frac{d_P}{a_{TF}}$$. 
Th latter it seems comes from figA15.3.

In [105]:
x1=1.6*u1/aBD
print(x1)
y1=d/aBD
print(y1)

1.33616055051
22.4498748399


Next we calculate 
$$FPS=FPS_oe^{-\frac{x'^p}{a}}$$
where 
$$a=4.3\left(1-e^{-\frac{y'^{1.1}}{12}}\right)$$,
$$p=0.4e^{\frac{-y'}{12}}+0.85$$
and
$$FPS_o=1.27\left(1-e^{-\frac{y'{0.76}}{3}}\right)$$.